In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

In [2]:
import pandas as pd

# Load the dataset into a pandas dataframe.
df = pd.read_csv("Data/test.csv")

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)

Number of training sentences: 11,490



,article,highlights,id
8481,It seems a late-night dinner at a fancy restau...,Locals were left wondering who'd done what aft...,b56e50294cadecb4a94acdf6fd0f63e5b930a868
135,(CNN)At least two people were taken into custo...,"Two people are taken into custody, but the pro...",1e1818bffffe5361e3ebe47358e9acf602f33f82
6341,Real Madrid closed the gap on Barcelona to jus...,Sergio Ramos opened the scoring for Real Madri...,801c81c8a38935f243f12e8271b3cf7efa02dc08
386,(CNN)Another controversy has erupted from the ...,Chris Evans and Jeremy Renner get in hot water...,568388111f9dd00ef144e7fc6a650b7d01700072
5424,Angelique Kerber of Germany upset three-time d...,Maria Sharapova lost to Angelique Kerber in St...,68c36241fa78a43645f5736c311e0dd755828951
254,"Dongguan, China (CNN)For a decade, the New Sou...","For a decade, the New South China Mall has lai...",3862305ca71a8f1c5823da03fcfb711beace5e63
9448,"Edwin 'Jock' Mee, (pictured outside court at a...","Edwin 'Jock' Mee, 45, allegedly targeted 11 ca...",cce33348c3c07a7cfc5fcb8ea4e3d69d096ccf42
5015,Holidaying with the lair of a tiger as your cl...,Natural World Safaris offers tourists the oppo...,5e5cdba2f7cae6fdcabfdd4e47481e362aa55449
319,(CNN)The verdict of the opinion polls on the U...,No clear winner in opinion polls after first U...,4568e6786809ad535892882ef6a346839de700cc
2106,Stephanie Scott was due to get married to her ...,Police discovered the body of a female in bush...,1805302601d24e43a5a49a3dd7cb605ef71ac85b


In [3]:
train_sentence = df.article.values
train_target = df.highlights.values

In [4]:
print(train_sentence.shape, train_target.shape)

(11490,) (11490,)


In [5]:
train_sentence = list(train_sentence)
train_target = list(train_target)

### Tokenize

In [8]:
from transformers import BartTokenizer, BartForConditionalGeneration
# Load the BART tokenizer.
print('Loading BART tokenizer...')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

/home/saiashish/.local/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


Loading BART tokenizer...


In [9]:
article_encoding = tokenizer(train_sentence, return_tensors='pt', padding=True, truncation = True)

In [10]:
article_input_ids = article_encoding['input_ids']
article_attention_mask = article_encoding['attention_mask']

In [11]:
print(article_input_ids.shape, article_attention_mask.shape)

torch.Size([11490, 1024]) torch.Size([11490, 1024])


In [12]:
target_encoding = tokenizer(train_target, return_tensors='pt', padding=True, truncation = True)

In [13]:
target_input_ids = target_encoding['input_ids']
target_attention_mask = target_encoding['attention_mask']

In [14]:
print(target_input_ids.shape, target_attention_mask.shape)

torch.Size([11490, 847]) torch.Size([11490, 847])


### Dataloader

In [15]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# The DataLoader needs to know our batch size for training, so we specify it 
# here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of
# 16 or 32.
batch_size = 16
# Create the DataLoader for our training set.
train_data = TensorDataset(article_input_ids, article_attention_mask, target_input_ids, target_attention_mask)

In [ ]:
train_sampler = RandomSampler(train_data)

In [18]:
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

### Loss

In [38]:
# The loss function
def loss_fn(lm_logits, labels):
    loss_fct = CrossEntropyLoss(ignore_index = tokenizer.pad_token_id)
    loss = loss_fct(lm_logits.view(-1, vocab_size), labels.view(-1))
    return loss
    

In [ ]:
# Import the model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
model.train()

In [ ]:
optimizer = torch.optim.SGD(model_decoder.bart_model.model.decoder.parameters(),1e-3,momentum=0.9,weight_decay=0.01)

In [ ]:
epochs = 500

for eps in range(epochs):
    # For each batch of training data...
    print('Epoch: ', eps)
    for step, batch in enumerate(train_dataloader):
        
        model.train()
        
        optimizer.zero_grad()
        
        out = model_decoder(input_ids=batch[0], output_ids=batch[2], encoder_attention_mask=batch[1], decoder_attention_mask=batch[3])
        
        loss = loss_fn(out.logits, target_ids)
    
        # For each batch of training data... print loss and the batch number
        print("Step: %d, Loss: %.3f" % (step, loss.item()))

        print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in out.logits.argmax(dim = 2)])
    
        loss.backward()
        optimizer.step()